# Evaluate and submit predictions

### Imports

In [1]:
import os
if 'notebooks' in os.getcwd(): os.chdir('../..')  # change to main directory
print('Working directory:', os.getcwd() )

Working directory: /scratch/snx3000/bp000429/submission/adrian_sensorium


In [2]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

from sensorium.utility import submission
from sensorium.utility.training import read_config

### Load configuration for model

In [3]:
config_file = 'notebooks/submission_m3/config_oneModel_m3.yaml'
config = read_config( config_file )
print(config)

ordereddict([('data_sets', ['notebooks/data/static27204-5-13-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip']), ('dataset_fn', 'sensorium.datasets.static_loaders'), ('dataset_config', ordereddict([('normalize', True), ('include_behavior', True), ('include_eye_position', True), ('batch_size', 128), ('scale', 0.25), ('preload_from_merged_data', True), ('include_trial_id', True), ('include_rank_id', True), ('include_history', True), ('include_behav_state', True), ('adjusted_normalization', True)])), ('model_fn', 'sensorium.models.modulated_stacked_core_full_gauss_readout'), ('model_seed', 3452), ('model_config', ordereddict([('pad_input', False), ('stack', -1), ('layers', 4), ('input_kern', 9), ('gamma_input', 9.8), ('gamma_readout', 0.48), ('hidden_kern', 10), ('hidden_channels', 64), ('depth_separable', True), ('grid_mean_predictor', ordereddict([('type', 'cortex'), ('input_dimensions', 2), ('hidden_layers', 4), ('hidden_features', 20), ('nonlinearity', 'ReLU'), ('final_tanh', True)]

### Prepare dataloader

In [4]:
if config['data_sets'][0] == 'all':
    basepath = "notebooks/data/"
    filenames = [os.path.join(basepath, file) for file in os.listdir(basepath) if ".zip" in file ]
    filenames = [file for file in filenames if 'static26872-17-20' not in file]
else:
    filenames = config['data_sets']
    # filenames like ['notebooks/data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]
    
dataset_fn = config['dataset_fn']  # 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                  **config['dataset_config'],
                 }

dataloaders = get_data(dataset_fn, dataset_config)

### Load saved model

In [5]:
# Instantiate model
model_fn = config['model_fn']     # e.g. 'sensorium.models.modulated_stacked_core_full_gauss_readout'
model_config = config['model_config']

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=config['model_seed'],
                 )

# Load trained weights
save_file = 'notebooks/submission_m3/results/model_v3_retrained.pth'
model.load_state_dict(torch.load(save_file));

### Generate submission file

In [6]:
dataset_name = '27204-5-13'

submission.generate_submission_file(trained_model=model, 
                                    dataloaders=dataloaders,
                                    data_key=dataset_name,
                                    path="notebooks/submission_m3/results_retrained/",
                                    device="cuda")

Submission file saved for tier: live_test. Saved in: notebooks/submission_m3/results_retrained/submission_file_live_test.csv
Submission file saved for tier: final_test. Saved in: notebooks/submission_m3/results_retrained/submission_file_final_test.csv


### Evaluate model on all datasets

In [7]:
from sensorium.utility import get_correlations, get_signal_correlations, get_fev
from sensorium.utility.measure_helpers import get_df_for_scores

#### Test data

In [8]:
single_trial_correlation = get_correlations(model, dataloaders, tier="test", device="cuda", as_dict=True)

df_test = get_df_for_scores(session_dict=single_trial_correlation,
                                    measure_attribute="Single Trial Correlation"
                      )

In [9]:
df_test.groupby('dataset').describe()

Single Trial Correlation                                   
                              count mean  std  min  25%  50%  75%  max
dataset                                                               
27204-5-13                   7538.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

#### Validation data

In [10]:
single_trial_correlation = get_correlations(model, dataloaders, tier="validation", device="cuda", as_dict=True)

df_val = get_df_for_scores(session_dict=single_trial_correlation,
                                    measure_attribute="Single Trial Correlation"
                      )

In [11]:
df_val.groupby('dataset').describe()

Single Trial Correlation                                          \
                              count      mean       std       min       25%   
dataset                                                                       
27204-5-13                   7538.0  0.451261  0.132818  0.033228  0.356523   

                                          
                 50%       75%       max  
dataset                                   
27204-5-13  0.450566  0.543477  0.850034